## Non-Interactive Authentication to OSDU

This notebook is intended to serve as introduction on how to send API requests to OSDU using an non-interactive OAuth 2.0 methodology.

What is OAuth?
OAuth 2.0, which stands for "Open Authorization," is a standard designed to allow a website or application to access resources hosted by other web applications on behalf of a user.

The notebooks covers the following points:

1. Sending GET and POST request using Microsoft Authentication Library (MSAL).
2. Sending GET and POST request using OSDU SDK.

##### 1. Sending GET and POST request using Microsoft Authentication Library (MSAL)

In [1]:
import msal
import requests, json
import os

# Adding .env file variables as environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
"""
The authentication/authorization requires an application to have been regitered within Azure under
EQN tenancy, which will provide the application with a client_id.

Once the previous is done the application will require to have access to the OSDU main principals
for DEV, TEST, and PROD, so that we can make requests to OSDU using its client_id.

The application can be either a PublicClientApplication or ConfidentialClientApplication.
We will use a ConfidentialClientApplication for this example to make a simple request to OSDU,
which allows us authenticate non-intectively by passing the client_id and client_secret as
arguments of the ConfidentialClientApplication method.
"""

tenant_id = os.environ["tenant_id"]
client_id = os.environ["osdu_np_client_id"]
client_secret = os.environ["osdu_np_client_secret"]
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = f'{os.environ["np_resource_id"]}/.default openid profile offline_access'


"""Then we instantiate a ConfidentialClientApplication."""

app_client = msal.ConfidentialClientApplication(
    client_id=client_id,
    client_credential = client_secret,
    authority=authority,
)


"""Then we request an access token using the app_client credentials, which will be send in the
body of our request."""

result = app_client.acquire_token_for_client(scopes=scopes)
access_token = result['access_token']

In [3]:

"""
GET REQUEST: Listing legal tags.

To make a request to OSDU, we need to append the access token and also the osdu data partition id
{PROD: "equinor-data", TEST: "npequinor-test", DEV: "npequinor-dev"} we aim to target. 
"""

osdu_np_server = os.environ["osdu_np_server"]

headers={
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    'Accept': 'application/json',
    "data-partition-id": os.environ["osdu_npdev_data_partition_id"],        
}


"""Sending a GET request"""

resp = requests.request(
    "GET",
    url = f"{osdu_np_server}/legal/v1/legaltags?valid=true",
    headers=headers
)


"""Printing first legal in the response"""

resp.json()['legalTags'][0]

{'name': 'npequinor-dev-equinor-private-default',
 'description': 'A default legaltag to be used for Equinor data',
 'properties': {'countryOfOrigin': ['NO'],
  'contractId': 'Unknown',
  'expirationDate': '9999-12-31',
  'originator': 'Equinor',
  'dataType': 'First Party Data',
  'securityClassification': 'Private',
  'personalData': 'No Personal Data',
  'exportClassification': 'No License Required'}}

In [4]:
"""
POST REQUEST: OSDU aggregated records by schema kind.

To make a request to OSDU, we need to append the access token and also the osdu data partition id
{PROD: "equinor-data", TEST: "npequinor-test", DEV: "npequinor-dev"} we aim to target. 
"""

osdu_np_server = os.environ["osdu_np_server"]

headers={
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "data-partition-id": os.environ["osdu_npdev_data_partition_id"],        
}

schema_kinds = "*:*:*:*"
payload = payload = {"kind": schema_kinds, 'aggregateBy': 'kind', 'query': "*"}


"""Sending a POST request"""

resp = requests.request(
    "POST",
    url = f"{osdu_np_server}/search/v2/query",
    data = json.dumps(payload),
    headers=headers
)


"""Printing first legal in the response"""

resp.json()['aggregations'][:5]

[{'key': 'eqnr:smda-api-v2.0:wellbore-alias:1.0.0', 'count': 747693},
 {'key': 'osdu:wks:master-data--Organisation:1.1.0', 'count': 125565},
 {'key': 'eqnr:smda-api-v2.0:wellheaders:1.0.0', 'count': 22546},
 {'key': 'osdu:wks:master-data--Wellbore:1.1.0', 'count': 22531},
 {'key': 'eqnr:iEnergy-sdb:seismicpoststackdatasets:1.0.0', 'count': 17685}]

##### 2. Sending GET and POST request using OSDU SDK.

In [5]:
from osdu.identity import OsduMsalNonInteractiveCredential
from osdu.client import OsduClient

In [6]:
"""
The authentication/authorization requires an application to have been regitered within Azure under
EQN tenancy, which will provide the application with a client_id.

Once the previous is done the application will require to have access to the OSDU main principals
for DEV, TEST, and PROD, so that we can make requests to OSDU using its client_id.

The application can be either a PublicClientApplication or ConfidentialClientApplication.
We will use a ConfidentialClientApplication for this example to make a simple request to OSDU,
which allows us authenticate non-intectively by passing the client_id and client_secret as
arguments of the ConfidentialClientApplication method.
"""

tenant_id = os.environ["tenant_id"]
client_id = os.environ["osdu_np_client_id"]
client_secret = os.environ["osdu_np_client_secret"]
authority = f"https://login.microsoftonline.com/{tenant_id}"
scopes = f'{os.environ["np_resource_id"]}/.default openid profile offline_access'


"""First we instantiate a ConfidentialClientApplication."""

app_private_client = msal.ConfidentialClientApplication(
    client_id=client_id,
    client_credential = client_secret,
    authority=authority,
)

"""Then we instantiate our credentials, which will allow us authenticate interactively to retrieve
and acceess token which will be passes latter in our requests to OSDU."""
credentials = OsduMsalNonInteractiveCredential(
    client_id=client_id,
    client_secret=client_secret,
    authority=authority,
    scopes=scopes,
    client=app_private_client
)

"""Then we instantiate our client by passing our credentials and the OSDU data partition we want to
make requests to."""
client = OsduClient(server_url=os.environ["osdu_np_server"],
                    data_partition=os.environ["osdu_npdev_data_partition_id"],
                    credentials=credentials
                    )

In [7]:
"""
GET REQUEST: Listing legal tags.

Then we make a GET request in the same way as we did before.
"""

osdu_np_server = os.environ["osdu_np_server"]

headers = client.get_headers()

resp = requests.request(
    "GET",
    url = f"{osdu_np_server}/legal/v1/legaltags?valid=true",
    headers=headers
)

resp.json()['legalTags'][0]

{'name': 'npequinor-dev-equinor-private-default',
 'description': 'A default legaltag to be used for Equinor data',
 'properties': {'countryOfOrigin': ['NO'],
  'contractId': 'Unknown',
  'expirationDate': '9999-12-31',
  'originator': 'Equinor',
  'dataType': 'First Party Data',
  'securityClassification': 'Private',
  'personalData': 'No Personal Data',
  'exportClassification': 'No License Required'}}

In [8]:
"""
POST REQUEST: OSDU aggregated records by schema kind.

Then we make a POST request in the same way as we did before.
"""

osdu_np_server = os.environ["osdu_np_server"]

schema_kinds = "*:*:*:*"
payload = payload = {"kind": schema_kinds, 'aggregateBy': 'kind', 'query': "*"}
headers = client.get_headers()

resp = requests.request(
    "POST",
    url = f"{osdu_np_server}/search/v2/query",
    data = json.dumps(payload),
    headers=headers
)

resp.json()['aggregations'][:5]

[{'key': 'eqnr:smda-api-v2.0:wellbore-alias:1.0.0', 'count': 747693},
 {'key': 'osdu:wks:master-data--Organisation:1.1.0', 'count': 125565},
 {'key': 'eqnr:smda-api-v2.0:wellheaders:1.0.0', 'count': 22546},
 {'key': 'osdu:wks:master-data--Wellbore:1.1.0', 'count': 22531},
 {'key': 'eqnr:iEnergy-sdb:seismicpoststackdatasets:1.0.0', 'count': 17685}]